In [ ]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
from typing import List, Dict

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_cytoscape as cyto


In [ ]:
load_dotenv('../featuring_network.env')
client = os.getenv('SPOTIFY_CLIENT', 'client')
secret = os.getenv('SPOTIFY_SECRET', 'secret')

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client, client_secret=secret))

In [ ]:
def get_artist_id(name: str) -> str:
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None
    
def get_artist_album_ids(artist_id: str) -> List:
    album_names = set()
    albums = set()
    results = sp.artist_albums(artist_id, album_type='album')
    for album in results['items']:
        if album['name'] not in album_names:
            albums.add(album['id'])
            album_names.add(album['name'])
    return list(albums)
    
# issues with edits 
def get_album_track_ids(album_ids: List, artist_id: str) -> Dict:
    songs_checker = set()
    songs_w_features = {}
    for album in album_ids:
        track_ids = sp.album_tracks(album)['items']
        for track in track_ids:
            features = [x['name'] for x in track['artists'] if x['uri'] != artist_id]
            # exclude songs w. no features
            if len(features)>1 and track['name'] not in songs_checker:
                songs_w_features[track['id']] = {}
                songs_w_features[track['id']]['name'] = track['name']
                songs_w_features[track['id']]['features'] = features
                songs_checker.add(track['name'])

    return songs_w_features
    
def count_appearances(tracks: Dict) -> Dict:
    features = {}
    for track in tracks:
        for artist in tracks[track]['features']:
            if artist not in features.keys():
                features[artist] = 1
            else:   
                features[artist] += 1
    return features

def make_node(entity_id, label):
    return {'data': {'id':entity_id, 'label':label}}
    
def make_edge(root, child, weight):
    return {'data': {'source': root, 'target': child, 'weight': weight}}
    
def generate_graph(feature_counter, artist_name):
    graph = [{'data': {'id':artist_name, 'label':artist_name}}]
    for feat in feature_counter:
        #generate node
        graph.append(make_node(feat, feat))
        #generate edge
        graph.append(make_edge(artist_name, feat, feature_counter[feat]))
        
    return graph

kanye_id = get_artist_id('Kanye West')['uri']
kanye_album_ids = get_artist_album_ids(kanye_id)
kanye_songs_w_features = get_album_track_ids(kanye_album_ids, kanye_id)
graph_data = generate_graph(count_appearances(kanye_songs_w_features), 'Kanye West')

In [ ]:

# Load Data
df = px.data.tips()# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.P("Dash Cytoscape:"),
    cyto.Cytoscape(
        id='cytoscape',
        elements=graph_data,
        layout={'name': 'random'},
        style={'width': '800px', 'height': '500px'}
    )
])

app.run_server(mode='inline')